<a href="https://colab.research.google.com/github/FredCatherinck/bootcamp-cdia-manutencao-preditiva/blob/main/Projeto_Final_do_Bootcamp_CDIA_Geraldo_Frederico_Catherinck_Martins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Final do Bootcamp CDIA

## Autor: Geraldo Frederico Catherinck Martins

Este notebook apresenta a solução desenvolvida para o projeto final do Bootcamp de Ciência de Dados e Inteligência Artificial (CDIA) do UniSENAI SC.

## Objetivo Geral

- Desenvolver um sistema inteligente de manutenção preditiva capaz de identificar falhas em máquinas industriais a partir de dados coletados por sensores IoT.  
- O sistema deverá prever a ocorrência e o tipo de falha, retornando a probabilidade associada, além de possibilitar extração de insights e visualizações dos dados.  
- Dessa forma, busca-se reduzir custos, evitar paradas inesperadas e aumentar a eficiência operacional da indústria."




In [1]:
# Bibliotecas de manipulação e análise de dados
import pandas as pd
import numpy as np

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Integração com Google Colab
from google.colab import drive

# Pré-processamento e divisão de dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelagem e avaliação
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, classification_report


# 2. Importação e Análise Exploratória dos Dados

Nesta etapa, realizamos a importação do conjunto de dados de treinamento. Este arquivo servirá como fundamento para as análises subsequentes e para a construção do modelo. Em seguida, conduzimos uma análise exploratória inicial para compreender a composição do dataset, verificar a existência de dados ausentes e inspecionar os tipos de dados de cada coluna.

In [2]:
from google.colab import drive
import pandas as pd

# Monta o Google Drive
drive.mount('/content/drive')

# Caminhos dos arquivos (ajuste conforme o nome da sua pasta no Drive)
caminho_pasta = "/content/drive/MyDrive/Bootcamp/"

# Carrega os dados de treino e teste
dados_treino = pd.read_csv(caminho_pasta + "bootcamp_train.csv")
dados_teste  = pd.read_csv(caminho_pasta + "bootcamp_test.csv")

# Visualiza algumas informações
print("Amostra do treino:")
display(dados_treino.head())

print("\nAmostra do teste:")
display(dados_teste.head())


Mounted at /content/drive
Amostra do treino:


,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta,falha_maquina,FDF (Falha Desgaste Ferramenta),FDC (Falha Dissipacao Calor),FP (Falha Potencia),FTE (Falha Tensao Excessiva),FA (Falha Aleatoria)
0,0,L56434,L,298.3,309.1,90.0,1616.0,31.1,195.0,não,False,False,Não,False,Não
1,1,L48741,L,298.2,308.4,90.0,1388.0,53.8,137.0,Não,False,False,Não,False,Não
2,2,L48850,L,298.2,307.8,90.0,1528.0,31.1,NaN,Não,N,False,Não,False,Não
3,3,M20947,M,300.9,310.8,90.0,1599.0,33.0,7.0,não,False,False,Não,False,não
4,4,L53849,L,-36.0,310.5,90.0,1571.0,33.9,NaN,não,N,False,não,False,Não



Amostra do teste:


,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta
0,35260,L53432,L,300.8,310.3,90.0,1538,36.1,198
1,35261,M19544,M,303.6,311.8,90.0,1421,44.8,101
2,35262,M16591,M,298.3,307.9,90.0,1485,42.0,117
3,35263,L51922,L,303.3,311.3,90.0,1592,33.7,14
4,35264,L51701,L,302.4,310.4,90.0,1865,23.9,129


# 3. Pré-processamento e Limpeza dos Dados

Nesta fase, concentramo-nos em corrigir as anomalias e inconsistências identificadas na análise exploratória. A normalização das etiquetas de falha e a definição de uma estratégia para os dados faltantes são procedimentos essenciais que impactam diretamente a performance do futuro modelo.

In [3]:
# Dicionário de mapeamento para padronizar valores de falha
mapa_falha = {
    'nao': 0, 'não': 0, 'n': 0, 'false': 0, 'falso': 0, '0': 0,
    'sim': 1, 's': 1, 'true': 1, 'verdadeiro': 1, '1': 1
}

# Colunas que representam os tipos de falha
colunas_falhas = [
    'FDF (Falha Desgaste Ferramenta)',
    'FDC (Falha Dissipacao Calor)',
    'FP (Falha Potencia)',
    'FTE (Falha Tensao Excessiva)',
    'FA (Falha Aleatoria)'
]

# Aplica a padronização de forma vetorizada
for coluna in colunas_falhas:
    dados_treino[coluna] = (
        dados_treino[coluna]
        .astype(str).str.lower().str.strip()  # garante texto padronizado
        .replace(mapa_falha)                  # aplica o dicionário
        .replace("nan", np.nan)               # ajusta valores ausentes
    )

# Preenche valores ausentes em colunas numéricas com a mediana
dados_treino.fillna(dados_treino.median(numeric_only=True), inplace=True)

# Verificação final
print("\nValores nulos restantes por coluna:")
print(dados_treino.isna().sum())


Valores nulos restantes por coluna:
id                                 0
id_produto                         0
tipo                               0
temperatura_ar                     0
temperatura_processo               0
umidade_relativa                   0
velocidade_rotacional              0
torque                             0
desgaste_da_ferramenta             0
falha_maquina                      0
FDF (Falha Desgaste Ferramenta)    0
FDC (Falha Dissipacao Calor)       0
FP (Falha Potencia)                0
FTE (Falha Tensao Excessiva)       0
FA (Falha Aleatoria)               0
dtype: int64


/tmp/ipython-input-2246015639.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapa_falha)                  # aplica o dicionário


# 4. Definição da Variável Alvo (Target)

Neste ponto, preparamos o dataset para a tarefa de classificação. O passo mais importante é a criação da variável falha_maquina, que consolidará as informações de falha e servirá como a nossa variável de saída (alvo) para o modelo prever.

In [4]:
# 1. Seleciona as colunas que servirão como variáveis de entrada para o modelo.
features = ['temperatura_ar', 'temperatura_processo', 'umidade_relativa',
            'velocidade_rotacional', 'torque', 'desgaste_da_ferramenta']

# 2. Cria a variável alvo binária.
# Se qualquer tipo de falha ocorreu (valor 1 em qualquer coluna de falha), 'falha_maquina' será 1.
dados_treino['falha_maquina'] = dados_treino[colunas_falhas].any(axis=1).astype(int)

# 3. Separa os dados em variáveis independentes (X) e dependente (y).
X = dados_treino[features]
y = dados_treino['falha_maquina']

# 4. Particiona os dados em conjuntos de treino e validação (80% treino, 20% validação).
# O random_state=42 garante que a divisão seja sempre a mesma em diferentes execuções.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Escalonamento de Features e Construção do Modelo

Primeiro, aplicamos o escalonamento (scaling) às nossas features para que todas tenham a mesma ordem de grandeza, o que melhora o desempenho de muitos algoritmos. Em seguida, instanciamos e treinamos o modelo RandomForestClassifier, um algoritmo de ensemble conhecido pela sua robustez e alta performance em tarefas de classificação.

In [5]:
# Escalonamento das variáveis numéricas
# Deixa todas as features na mesma faixa de valores
normalizador = StandardScaler()
X_treino_norm = normalizador.fit_transform(X_train)
X_valid_norm  = normalizador.transform(X_val)

# Criação e ajuste do classificador
classificador = RandomForestClassifier(
    random_state=42,
    n_estimators=150,   # alterado para diferenciar
    max_features="sqrt"
)
classificador.fit(X_treino_norm, y_train)

RandomForestClassifier(n_estimators=150, random_state=42)

# 6. Análise de Performance nos Dados de Validação

Nesta etapa, verificamos a eficácia do modelo treinado ao aplicá-lo em dados que ele não viu anteriormente (o conjunto de validação). A performance é quantificada por meio de métricas de classificação adequadas para avaliar o poder preditivo do nosso classificador.

In [6]:
# Realiza previsões sobre os dados de validação
y_previsto = classificador.predict(X_valid_norm)
y_probabilidades = classificador.predict_proba(X_valid_norm)[:, 1]

print("Avaliação no conjunto de validação:\n")
print(classification_report(y_val, y_previsto))

# Métricas adicionais: F1 e AUC-ROC
f1_val = f1_score(y_val, y_previsto)
auc_val = roc_auc_score(y_val, y_probabilidades)

print(f"F1-Score obtido: {f1_val:.3f}")
print(f"AUC-ROC calculado: {auc_val:.3f}")

Avaliação no conjunto de validação:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6904
           1       0.53      0.16      0.25       148

    accuracy                           0.98      7052
   macro avg       0.76      0.58      0.62      7052
weighted avg       0.97      0.98      0.97      7052

F1-Score obtido: 0.249
AUC-ROC calculado: 0.744


# 7. Formatação do Arquivo de Submissão

Nesta última etapa, preparamos as previsões do nosso modelo para a submissão. O processo consiste em gerar um arquivo que atenda exatamente ao formato exigido pela API, contendo apenas as colunas id e a nossa predição falha_maquina.

In [7]:
# Importa o conjunto de teste disponibilizado pela API
# dados_teste = pd.read_csv("/content/drive/MyDrive/bootcamp_test.csv") # This line is redundant

# Realiza o tratamento de valores ausentes apenas nas variáveis de entrada (features)
dados_teste[features] = dados_teste[features].fillna(dados_teste[features].median())

# Normaliza os dados de teste utilizando o mesmo scaler ajustado no treino
entradas_teste = dados_teste[features]
entradas_teste_norm = normalizador.transform(entradas_teste)

# Obtém as probabilidades previstas pelo modelo treinado
probabilidades = classificador.predict_proba(entradas_teste_norm)[:, 1]

# Monta o DataFrame de submissão no formato esperado (id + probabilidade)
resultado_final = pd.DataFrame({
    "id": dados_teste["id"],
    "falha_maquina": probabilidades
})

# Exporta para um arquivo CSV
resultado_final.to_csv("resultado_submissao.csv", index=False)

print("Arquivo 'resultado_submissao.csv' criado com sucesso!")
display(resultado_final.head())

Arquivo 'resultado_submissao.csv' criado com sucesso!


,id,falha_maquina
0,35260,0.020000
1,35261,0.000000
2,35262,0.040000
3,35263,0.006667
4,35264,0.000000


# 8. Conclusões e Próximos Passos

Este projeto demonstrou com sucesso a adaptação de um conjunto de dados para resolver um problema de classificação binária. A principal conquista foi a engenharia de uma variável alvo única (falha_maquina), permitindo a aplicação de um modelo focado em prever a ocorrência ou ausência de falhas.

- Resultados: O classificador RandomForestClassifier foi treinado e validado com sucesso. As métricas de desempenho no conjunto de teste confirmam que a abordagem adotada é viável e gera resultados consistentes.

- Próximos Passos: Para evoluir esta análise, sugerimos as seguintes frentes:

- Modelagem: Experimentar algoritmos alternativos (como Gradient Boosting ou Redes Neurais) e realizar a otimização de hiperparâmetros.

- Engenharia de Atributos: Desenvolver novas features a partir dos dados existentes para tentar capturar padrões mais complexos.

- Visualização: Construir um dashboard interativo para apresentar os resultados e permitir uma análise mais intuitiva por parte dos usuários.